In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch import cuda
pin_memory=False 

C:\Users\alway\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.cuda.is_available())

# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(torch.cuda.get_device_name(cuda_id))


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#pil verison fix
import PIL

from PIL import __version__
print(__version__)

from PIL import __version__
PIL.PILLOW_VERSION = __version__

print(PIL.PILLOW_VERSION)


True
NVIDIA GeForce GTX 1050 Ti with Max-Q Design
cuda
9.4.0
9.4.0


In [3]:
from resnet_pytorch import ResNet 
model = ResNet.from_pretrained('resnet18', num_classes=10)

Loaded pretrained weights for resnet18.


In [ ]:
train_dir = 'C:/Users/alway/Documents/GitHub/ML-Biomed/Data/train'
test_dir = 'C:/Users/alway/Documents/GitHub/ML-Biomed/Data/test'
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)